In [1]:
#import libraries
import time
import numpy as np
import pandas as pd
from scipy.io import loadmat
import csv
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
%gui qt5
import KMedoids
from sklearn.metrics.pairwise import pairwise_distances

import matplotlib.pyplot as plt

from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

### Plotting

In [ ]:
#set up plottig GUI
app = QtGui.QApplication([])
pg.setConfigOption('background','w')

In [ ]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')
plot2 = win.addPlot()
plot2.setXRange(-6,6)
plot2.setYRange(0,6)
plot2.setLabel('left',text = 'Y position (m)')
plot2.setLabel('bottom', text= 'X position (m)')
s2 = plot2.plot([],[],pen=None,symbol='o')
plot3 = win.addPlot()
plot3.setXRange(-6,6)
plot3.setYRange(0,6)
plot3.setLabel('left',text = 'Y position (m)')
plot3.setLabel('bottom', text= 'X position (m)')
s3 = plot3.plot([],[],pen=None,symbol='o')

### Parsing

In [ ]:
def tlvParsing(data, tlvHeaderLengthInBytes, pointLengthInBytes, targetLengthInBytes):
    
    data = np.frombuffer(data, dtype = 'uint8')
    
    targetDict = dict()
    pointCloud = np.array([])
    index = 0
    #tlv header parsing
    tlvType = data[index:index+4].view(dtype=np.uint32)
    tlvLength = data[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    pointCloudDataLength = tlvLength - tlvHeaderLengthInBytes
    if tlvType.size > 0 and tlvType == 6: #point cloud TLV
        numberOfPoints = pointCloudDataLength/pointLengthInBytes
#         print('NUMBER OF POINTS ', str(int(numberOfPoints)))
        if numberOfPoints > 0:
            p = data[index:index+pointCloudDataLength[0]].view(dtype=np.single)
            #form the appropriate array 
            #each point is 16 bytes - 4 bytes for each property - range, azimuth, doppler, snr
            pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")
    
    #increment the index so it is possible to read the target list
    index += pointCloudDataLength
    #tlv header parsing
    tlvType = data[index[0]:index[0]+4].view(dtype=np.uint32)
    tlvLength = data[index[0]+4:index[0]+8].view(dtype=np.uint32)
    index += tlvHeaderLengthInBytes
    targetListDataLength = tlvLength - tlvHeaderLengthInBytes
    if tlvType.size > 0 and tlvType == 7: #target List TLV
        
        numberOfTargets = targetListDataLength/targetLengthInBytes
        TID = np.zeros((1, int(numberOfTargets[0])), dtype = np.uint32) #tracking IDs
        kinematicData = np.zeros((6, int(numberOfTargets[0])), dtype = np.single)
        errorCovariance = np.zeros((9, int(numberOfTargets[0])), dtype = np.single)
        gatingGain = np.zeros((1, int(numberOfTargets[0])), dtype = np.single)
        
        #increment the index so it is possible to read the target list
        targetIndex = 0
        while targetIndex != int(numberOfTargets[0]):
            TID[0][targetIndex] = data[index[0]:index[0]+4].view(dtype=np.uint32)
            kinematicData[:,targetIndex] = data[index[0]+4:index[0]+28].view(dtype=np.single)
            errorCovariance[:,targetIndex] = data[index[0]+28:index[0]+64].view(dtype=np.single)
            gatingGain[:,targetIndex] = data[index[0]+64:index[0]+68].view(dtype=np.single)
            index += targetLengthInBytes
            targetIndex += 1
            
        targetDict['TID'] = TID
        targetDict['kinematicData'] = kinematicData
        targetDict['errorCovariance'] = errorCovariance
        targetDict['gatingGain'] = gatingGain
    
    return pointCloud, targetDict

def parsePointCloud(pointCloud): #remove points that are not within the boundary
    
    effectivePointCloud = np.array([])
    
    for index in range(0, len(pointCloud[0,:])):
        if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
        and (pointCloud[1, index] > -50*np.pi/180 \
            and pointCloud[1, index] < 50*np.pi/180):

            #concatenate columns to the new point cloud
            if len(effectivePointCloud) == 0:
                effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
            else:
                point = np.reshape(pointCloud[:, index], (4,1),order="F")
                effectivePointCloud = np.hstack((effectivePointCloud, point))

    if len(effectivePointCloud) != 0:
        posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
        posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
        SNR  = effectivePointCloud[3,:]
    
        return posX,posY,SNR
    

### Clustering

In [ ]:
def iterativeDfs(vertexID, edgeMatrix, startNode):
    
    visited = np.array([], dtype=np.int)
    dfsStack = np.array([startNode])

    while dfsStack.size > 0:
        vertex, dfsStack = dfsStack[-1], dfsStack[:-1] #equivalent to stack pop function
        if vertex not in visited:
            #find unvisited nodes
            unvisitedNodes = vertexID[np.logical_not(np.isnan(edgeMatrix[int(vertex), :]))]
            visited = np.append(visited, vertex)
            #add unvisited nodes to the stack
            dfsStack = np.append(dfsStack, unvisitedNodes[np.logical_not(np.isin(unvisitedNodes,visited))])
    
    return visited

In [ ]:
def TreeClusteringOnly(posX, posY, SNR, weightThreshold, minClusterSize):
    
    vertexID = np.arange(len(posX))
    vertexList = np.arange(len(posX))

    associatedPoints = np.array([])

    if len(posX) >= minClusterSize:
        edgeMatrix = np.zeros((len(posX), len(posY)))

        #create distance matrix
        #x1 - x0
        xDifference = np.subtract(np.repeat(posX, repeats=len(posX)).reshape(len(posX), len(posX)), 
                                  np.transpose(np.repeat(posX, repeats=len(posX)).reshape(len(posX), len(posX))))
        #y1 - y0
        yDifference = np.subtract(np.repeat(posY, repeats=len(posY)).reshape(len(posY), len(posY)), 
                                  np.transpose(np.repeat(posY, repeats=len(posY)).reshape(len(posY), len(posY))))
        #euclidean distance calculation
        edgeMatrix = np.sqrt(np.add(np.square(xDifference), np.square(yDifference)))

        #weight based reduction of graph/remove edges by replacing edge weight by np.NaN
        weightMask = np.logical_or(np.greater(edgeMatrix,weightThreshold), np.equal(edgeMatrix, 0))
        edgeMatrix[weightMask] = np.NaN

        #perform iterative dfs
        associatedPoints = np.array([])
        
        
        centroidNumber = 0
        while vertexID.size > 0:
            startNode = vertexID[0]
            visited = iterativeDfs(vertexList, edgeMatrix, startNode)
            #remove visited nodes (ie only slice off all unvisited nodes)
            vertexID = vertexID[np.logical_not(np.isin(vertexID, visited))]
#             #visited is a component, extract cluster from it if possible
            if visited.size >= minClusterSize:
                cluster =  np.array([posX[visited], posY[visited],SNR[visited],
                                     np.repeat(centroidNumber, repeats=len(visited))])
                if associatedPoints.size == 0:
                    associatedPoints = cluster
                else:
                    associatedPoints = np.hstack((associatedPoints, cluster))
                centroidNumber += 1
                                    
                


    return associatedPoints

### Tracking

In [ ]:
def DKF(x, P, R, Q=0., dt=1.0):
    """ Returns a KalmanFilter which implements a
    constant velocity model for a state [x dx].T
    """
    
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.x = np.array([x[0], x[1]]) # location and velocity
    kf.F = np.array([[1., dt],
                     [0.,  1.]])  # state transition matrix
    kf.H = np.array([[1., 0]])    # Measurement function
    kf.R *= R                     # measurement uncertainty
    if np.isscalar(P):
        kf.P *= P                 # covariance matrix 
    else:
        kf.P[:] = P               # [:] makes deep copy
    if np.isscalar(Q):
        kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q)
    else:
        kf.Q[:] = Q
    return kf

### Main Code

In [ ]:
parsingMatFile = 'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Experiment Data 2\\3PeopleWalking.mat'

tlvData = (loadmat(parsingMatFile))['tlvStream'][0]

#RKF 
centroidX =np.zeros((4,1))
centroidP = []
P = np.identity(4);
centroidP.extend([P])

#DKF
#initialize variables
deltaT = 50*10**-3 #50ms

#system matrix
A = np.array([
    [1, deltaT, 0,0],
    [0,1,0,0],
    [0,0,1,deltaT],
    [0,0,0,1]
])

#output matrix
H = np.array([[1,0,0,0],
              [0,0,1,0]])
#covariance matrices
Q = np.eye(4)
R = np.ones(2).reshape(-1,1)
Pc = np.eye(4)

Pd = np.add(np.matmul(np.matmul(A,Pc), np.transpose(A)), Q) #prediction covariance
S = np.add(R, np.matmul(np.matmul(H, Pd), np.transpose(H))) #innovation covariance
K = np.matmul(Pd, np.matmul(np.transpose(H), np.linalg.inv(S))) #kalman gain
Pupdate = np.subtract(Pd, np.matmul(np.matmul(K, S), np.transpose(K))) #update covariance


#tree based
weightThresholdIntial = 0.2 #minimum distance between points
minClusterSizeInitial = 10
weightThresholdFinal = 0.8 #minimum distance between points
minClusterSizeFinal = 8 


#zone snr
snrFirstZone = 10
snrMiddleZone = 15
snrLastZone = 5

tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
targetLengthInBytes = 68

#optimization
expectedClustering = np.array([])
expectedKalman = np.array([])

s1.setData([],[])
s2.setData([],[])
s3.setData([],[])
s4.setData([],[])
QtGui.QApplication.processEvents()

tiPosX = np.array([])
tiPosY = np.array([])

Q = np.multiply(0.2,np.identity(4))
R = np.multiply(5,np.array([[1],[1]]))

for tlvStream in tlvData:
    
    #parsing
    pointCloud, targetDict = tlvParsing(tlvStream, tlvHeaderLengthInBytes, pointLengthInBytes, targetLengthInBytes)
    
    if pointCloud.size > 0:
        posX,posY,SNR = parsePointCloud(pointCloud) #dictionary that contains the point cloud data
        s1.setData(posX, posY)
        #initial noise reduction
        clusters = TreeClusteringOnly(posX, posY, SNR,
                                      weightThresholdIntial, minClusterSizeInitial)
        
        if clusters.size > 0:
            
            
#             row 1 - x
#             row 2 - y
#             row 3 - SNR
#             row 4 - cluster number
            
#             snr zone snr test
#             4.5 to the end -> last zone
#             3-4.5m -> middle zone
#             1-3m -> first zone
            snrMask_LastZone = np.logical_and(np.greater(clusters[1,:], 4.5), np.greater(clusters[2,:], snrLastZone)) #zone 4.5m and greater
            snrMask_MiddleZone = np.logical_and(np.logical_and(np.greater(clusters[1,:], 3), np.less_equal(clusters[1,:], 4.5)), 
                                                np.greater(clusters[2,:], snrMiddleZone)) #zone 3-4.5m with SNR > 20
            snrMask_FirstZone = np.logical_and(np.less_equal(clusters[1,:], 3), np.greater(clusters[2,:], snrFirstZone))
            overallSnrMask = np.logical_or(np.logical_or(snrMask_FirstZone,snrMask_MiddleZone), snrMask_LastZone)

            snrFilteredClusters = clusters[:,overallSnrMask]

            if snrFilteredClusters.size > 0:
                s2.setData(snrFilteredClusters[0,:], snrFilteredClusters[1,:])
                
                dbClusters = TreeClusteringOnly(snrFilteredClusters[0,:], snrFilteredClusters[1,:], 
                                                snrFilteredClusters[2,:], 
                                                weightThresholdFinal, minClusterSizeFinal)
                if dbClusters.size > 0:
                    #row 1 - x
                    #row 2 - y
                    #row 3 - cluster number
                    k = int(max(dbClusters[3,:])) + 1 
                    points = np.transpose(np.array([dbClusters[0,:], dbClusters[1,:]]))
                  
                    #kmeans 
                    centroidClusterer = KMeans(n_clusters= k).fit(points)
                    centroidData = np.array([centroidClusterer.cluster_centers_[:,0], centroidClusterer.cluster_centers_[:,1]])

                    #tracking
                    centroidX, centroidP = LiveRKF(centroidData, centroidX, centroidP, Q, R)
                    #plot
                    #calculate x and y positions
                    xPositions = np.multiply(centroidX[0,:], np.cos(centroidX[2,:]))
                    yPositions = np.multiply(centroidX[0,:], np.sin(centroidX[2,:]))
                   
                    #calculate range and theta to remove from graphing
#                     r = np.sqrt(np.add(np.square(xPositions),np.square(yPositions)))
#                     theta = np.rad2deg(np.arctan(np.divide(yPositions,xPositions)))
#                     positionMask = np.logical_and(np.less_equal(r,6), np.logical_and(np.less_equal(theta, 50), np.greater_equal(theta, -50)))
                    
#                     xPositions = xPositions[positionMask]
#                     yPositions = yPositions[positionMask]
                    
                    s3.setData(xPositions, yPositions)

    
    if len(targetDict) != 0:
        #kinematic data object structure
        #row 0 - posX
        #row 1 - posY 
        #row 2 - velX
        #row 3 - velY
        #row 4 - accX
        #row 5 - accY
        tiPosX = targetDict['kinematicData'][0,:]
        tiPosY = targetDict['kinematicData'][1,:]
        s4.setData(tiPosX,tiPosY)
        
    QtGui.QApplication.processEvents()
    k = input('ENTER ')
    
#     time.sleep(0.05)
    
#     currentClustering = input('Expected Clustering: ')
#     currentKalman = int(input('Expected Kalman: '))
#     expectedClustering = np.append(expectedClustering,currentClustering)
#     expectedKalman = np.append(expectedKalman,currentKalman)
    

#create pandas an output as csv
groundTruthDf = pd.DataFrame({'expectedClustering':expectedClustering, 'expectedKalman':expectedKalman})
groundTruthDf.to_csv('3PeopleWalkingGroundTruth.csv')